In [1]:
from __future__ import print_function
from bs4 import BeautifulSoup
import requests

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
}
 
# from 2018 season
url = 'https://www.footballdb.com/statistics/nfl/player-stats/kicking/2020/regular-season?sort=kickpts&limit=100'
req = requests.get(url, headers=headers)
soup = BeautifulSoup(req.text, 'html.parser')

print(soup)

In [21]:
# content
content = soup.find(id='leftcol')
# headers
headers = content.find_all('th')
# getting column names
cols = [head.text.strip() for head in headers if head.text.strip()]
print(cols)

['Player', 'Team', 'Gms', 'PAT', 'FG', '0-19', '20-29', '30-39', '40-49', '50+', 'Lg', 'Pts']


In [22]:
import pandas as pd
# stats
table = content.find_all('tr')
# converting to dataframe
res = []
for tr in table:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
# data
data = pd.DataFrame(res, columns=cols)
data.head()

,Player,Team,Gms,PAT,FG,0-19,20-29,30-39,40-49,50+,Lg,Pts
0,Daniel CarlsonD. Carlson,LV,16,45/47,33/35,0/0,18/18,8/8,3/5,4/4,54,144
1,Younghoe KooY. Koo,ATL,15,33/36,37/39,0/0,10/10,11/12,8/9,8/8,54,144
2,Jason SandersJ. Sanders,MIA,16,36/36,36/39,1/1,8/8,7/7,12/14,8/9,56,144
3,Tyler BassT. Bass,BUF,16,57/59,28/34,1/1,9/9,6/9,8/9,4/6,58,141
4,Rodrigo BlankenshipR. Blankenship,IND,16,43/45,32/37,0/0,10/10,12/13,9/11,1/3,53,139


In [23]:
# checking element type
data["Gms"] = pd.to_numeric(data["Gms"])
data["Lg"] = pd.to_numeric(data["Lg"])
data["Pts"] = pd.to_numeric(data["Pts"])

In [24]:
def split(csv,col):
  # splitting
  split = csv[col].str.split("/", expand=True)
  # adding
  csv[col] = pd.to_numeric(split[0])/pd.to_numeric(split[1])
  return

In [25]:
print("shape before: ",data.shape)
data = data.fillna(data.mean())
print("shape after: ",data.shape)
data.head()

shape before:  (47, 12)
shape after:  (47, 12)


<ipython-input-25-08e2fee29b26>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data = data.fillna(data.mean())


,Player,Team,Gms,PAT,FG,0-19,20-29,30-39,40-49,50+,Lg,Pts
0,Daniel CarlsonD. Carlson,LV,16,45/47,33/35,0/0,18/18,8/8,3/5,4/4,54,144
1,Younghoe KooY. Koo,ATL,15,33/36,37/39,0/0,10/10,11/12,8/9,8/8,54,144
2,Jason SandersJ. Sanders,MIA,16,36/36,36/39,1/1,8/8,7/7,12/14,8/9,56,144
3,Tyler BassT. Bass,BUF,16,57/59,28/34,1/1,9/9,6/9,8/9,4/6,58,141
4,Rodrigo BlankenshipR. Blankenship,IND,16,43/45,32/37,0/0,10/10,12/13,9/11,1/3,53,139


In [ ]:
import pandas as pd
# stats
table = content.find_all('tr')
# converting to dataframe
res = []
for tr in table:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

# data
data = pd.DataFrame(res, columns=cols)
data.head()

# renaming 
pivot_data = pd.DataFrame(res, columns=cols)
pivot_data.rename(columns={"0-19": "19", "20-29": "29", "30-39":"39","40-49":"49","50+":"59"}, inplace=True)

# numerical
csv = pivot_data
split(csv,"PAT")
split(csv,"FG")
split(csv,"19")
split(csv,"29")
split(csv,"39")
split(csv,"49")
split(csv,"59")
# filling

# filtering
pivot_data = pivot_data[['19','29','39','49','59']]

pivot_data.fillna(pivot_data.mean(), inplace=True)

pivot_data.head()

distance = []
kicks = []
# pivoting
for column in pivot_data:
  for row in range(len(pivot_data)):
    # adding data
    distance.append(int(column))
    kicks.append(pivot_data[column][row])

final = pd.DataFrame()
final['Distance'] = distance
final['Kicks'] = kicks

final.head()

**model 1**

In [27]:
from sklearn import linear_model
import numpy as np

# features and labels
X = np.array(final['Distance']).reshape((-1, 1))
y = final['Kicks']

# model
model = linear_model.LinearRegression()
model.fit(X, y)

# input
predict = 50

# predicting
predict =  np.array([predict]).reshape((-1, 1))
prediction= model.predict(predict)

# printing
print("kick success likeliness: ",prediction[0])

kick success likeliness:  0.7338397014678562


**model 2**

In [28]:
import statsmodels.api as sm
import numpy as np

# Create some example data
X = np.array(final['Distance']).reshape((-1, 1))
y = final['Kicks']

# Fit a logistic regression model
model = sm.GLM(y, sm.add_constant(X), family=sm.families.Binomial())
result = model.fit()

# Print the model summary
print(result.summary())

# 

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Kicks   No. Observations:                  235
Model:                            GLM   Df Residuals:                      233
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -67.935
Date:                Mon, 20 Mar 2023   Deviance:                       65.580
Time:                        20:15:52   Pearson chi2:                     84.5
No. Iterations:                     6   Pseudo R-squ. (CS):             0.1448
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.7440      0.907      6.333      0.0

In [ ]:
# 2018: 6.7863, -0.1102
# 2019: 6.3304, -0.1060
# 2020: 5.7440, -0.0915

**plotting**

In [48]:
import plotly.graph_objects as go
import plotly.express as px
from skimage import io
import numpy as np

x = np.linspace(0,70)
y = lambda x,b0,b1: np.exp(b0+b1*x)/(1+np.exp(b0+b1*x))

fig = go.Figure()

# fig = px.line(x=x, 
#              y=[y(x,6.7863,-0.1102),y(x,6.3304,-0.1060),y(x,5.7440,-0.0915)],
#              labels={'x':'Distance',
#                      'y':'Probability of Success'},
#              width=600, 
#              height=400
#             )

fig.add_trace(go.Scatter(
    x=x,
    y=y(x,6.7863,-0.1102),
    name="2018",
))

fig.add_trace(go.Scatter(
    x=x,
    y=y(x,6.3304,-0.1060),
    name="2019",
))

fig.add_trace(go.Scatter(
    x=x,
    y=y(x,5.7440,-0.0915),
    name="2020",
))

fig.update_layout(
    width=600, 
    height=400,
    xaxis_title="Distance",
    yaxis_title="Probability of Success",
)

fig.show()